## Wrangle and Analyze Data

In this project a dataset from Twitter profile WeRateDogs will be wrangled (gather, assess and clean). All the efforts for this task are going to be reported in this notebook. 

### Name: Daniel Guarino

## Table of Contents

- [Introduction](#intro)
- [Basic Libraries](#libraries)
- [Part I - Gathering Data](#gathering)
- [Part II - Assessing Data](#assess)
- [Part III - Cleaning Data](#regression)


<a id='intro'></a>
### Introduction

Wrangling data is very important.....

<a id='libraries'></a>
### Import Basic Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

<a id='gathering'></a>
### Part I - Gathering Data

**Reading CSV file (manually downloaded from 'Resources' folder on Udacity - Wrangle and Analyze Data web page)**

In [2]:
df = pd.read_csv('twitter_archive/twitter-archive-enhanced.csv')

In [3]:
df.head(1)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None


**Downloading a file programmatically**

In [4]:
import requests
import os

In [5]:
folder_name = 'twitter_archive'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)
response

<Response [200]>

In [6]:
with open(os.path.join(folder_name, 
                       url.split('/')[-1]), mode = 'wb') as file:
    file.write(response.content)

In [7]:
os.listdir(folder_name)

['twitter-archive-enhanced.csv', 'image-predictions.tsv', 'tweet_json.txt']

In [8]:
df_image = pd.read_csv('twitter_archive/image-predictions.tsv', sep='\t')
df_image.head(3)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True


**Importing and accessing Twitter API (tweep)**

In [9]:
import tweepy
from tweepy import OAuthHandler
import json
from timeit import default_timer as timer

In [11]:
consumer_key = 'HIDDEN'
consumer_secret = 'HIDDEN'
access_token = 'HIDDEN'
access_secret = 'HIDDEN'

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit = True)

In [12]:
tweet_ids = df.tweet_id.values
len(tweet_ids)

2356

In [ ]:
# Query Twitter's API for JSON data for each tweet ID in the Twitter archive
count = 0
fails_dict = {}
start = timer()
# Save each tweet's returned JSON as a new line in a .txt file
with open('twitter_archive/tweet_json.txt', 'w') as outfile:
    # This loop will likely take 20-30 minutes to run because of Twitter's rate limit
    for tweet_id in tweet_ids:
        count += 1
        print(str(count) + ": " + str(tweet_id))
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            print("Success")
            json.dump(tweet._json, outfile)
            outfile.write('\n')
        except tweepy.TweepError as e:
            print("Fail")
            fails_dict[tweet_id] = e
            pass
end = timer()
print(end - start)
print(fails_dict)

# Cell output hidden to save lines

In [10]:
# Converting relevant JSON data to dataframe

tweets_list =[]

with open('twitter_archive/tweet_json.txt') as json_file:
    for line in json_file:
    
        tweets_dict = {}
        tweets_json = json.loads(line)
        
        try:
            tweets_dict['tweet_id'] = tweets_json['extended_entities']['media'][0]['id']
        except:
            tweets_dict['tweet_id'] = 'na'

        tweets_dict['retweet_count'] = tweets_json['retweet_count']
        tweets_dict['favorite_count'] = tweets_json['favorite_count']
        
        tweets_list.append(tweets_dict)


In [11]:
df_api = pd.DataFrame(tweets_list)
df_api.head()

,tweet_id,retweet_count,favorite_count
0,892420639486877696,7947,37149
1,892177413194625024,5902,31957
2,891815175371796480,3902,24066
3,891689552724799489,8099,40478
4,891327551943041024,8785,38706


<a id='assess'></a>
### Part II - Assessing Data

### 1. Downloaded data by WeRateDogs page data frame info ###

In [12]:
df.head(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [14]:
df.dtypes

tweet_id                        int64
in_reply_to_status_id         float64
in_reply_to_user_id           float64
timestamp                      object
source                         object
text                           object
retweeted_status_id           float64
retweeted_status_user_id      float64
retweeted_status_timestamp     object
expanded_urls                  object
rating_numerator                int64
rating_denominator              int64
name                           object
doggo                          object
floofer                        object
pupper                         object
puppo                          object
dtype: object

In [15]:
# is there any duplicated ID?
sum(df['tweet_id'].duplicated())

0

In [16]:
# is there any duplicated name?
sum(df['name'].duplicated())

1399

In [17]:
# accuracy problems - 
sum(df['rating_numerator'] < df['rating_denominator'])

442

In [18]:
sum(df['rating_denominator'] != 10)

23

### Quality Issues ###

**1- Completeness:**

- Missing data at `expanded_urls` column
- Missing data at `name` column. Not every dog has a name in this dataset
- We pratically don't have data for `in_reply_to_status_id` , `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id`, `retweeted_status_timestamp` columns. These 05 columns are useless for this dataset. 

**2- Validity:**

- Column `tweet_id` should be a string variable instead integer
- Column `timestamp` should be a datetime variable instead string

**3 - Accuracy:**

- The `rating_numerator` column values should be always larger or equal the `rating_denominator` column values (= 10). This is not true for 442 records
- The `rating_denominator` column values should be always 10, but this is not true for 23 records

**4- Consistency:**

- This dataset misses a column with the division `rating_numerator` / `rating_denominator`. This column is really important to further analysis


### Tidiness Issues ###

- The `timestamp` column should have separated columns for date and hour
- We should have an unique column for Dog Stages with values like doggo, puppo, pupper, flooter or Nan (not every dog has this classification).

### 2. Downloaded programmatically neural network udacity image dataframe info ###

In [19]:
df_image.head(3)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True


In [20]:
df_image.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [21]:
df_image.img_num.value_counts()

1    1780
2     198
3      66
4      31
Name: img_num, dtype: int64

In [22]:
df_image.dtypes

tweet_id      int64
jpg_url      object
img_num       int64
p1           object
p1_conf     float64
p1_dog         bool
p2           object
p2_conf     float64
p2_dog         bool
p3           object
p3_conf     float64
p3_dog         bool
dtype: object

In [23]:
sum(df_image.duplicated())

0

In [57]:
# check how many breeds were predicted as a dog breed
sum(df_image['p1_dog'] == True)

1532

In [58]:
# check how many breeds were predicted as not a dog breed
sum(df_image['p1_dog'] == False)

543

In [26]:
df_image[df_image['p1_dog'] == False].head(10)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,0.045885,False,terrapin,0.017885,False
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,shopping_cart,0.962465,False,shopping_basket,0.014594,False,golden_retriever,0.007959,True
17,666104133288665088,https://pbs.twimg.com/media/CT56LSZWoAAlJj2.jpg,1,hen,0.965932,False,cock,0.033919,False,partridge,0.000052,False
18,666268910803644416,https://pbs.twimg.com/media/CT8QCd1WEAADXws.jpg,1,desktop_computer,0.086502,False,desk,0.085547,False,bookcase,0.079480,False
21,666293911632134144,https://pbs.twimg.com/media/CT8mx7KW4AEQu8N.jpg,1,three-toed_sloth,0.914671,False,otter,0.015250,False,great_grey_owl,0.013207,False
22,666337882303524864,https://pbs.twimg.com/media/CT9OwFIWEAMuRje.jpg,1,ox,0.416669,False,Newfoundland,0.278407,True,groenendael,0.102643,True
25,666362758909284353,https://pbs.twimg.com/media/CT9lXGsUcAAyUFt.jpg,1,guinea_pig,0.996496,False,skunk,0.002402,False,hamster,0.000461,False
29,666411507551481857,https://pbs.twimg.com/media/CT-RugiWIAELEaq.jpg,1,coho,0.404640,False,barracouta,0.271485,False,gar,0.189945,False
33,666430724426358785,https://pbs.twimg.com/media/CT-jNYqW4AAPi2M.jpg,1,llama,0.505184,False,Irish_terrier,0.104109,True,dingo,0.062071,False
43,666776908487630848,https://pbs.twimg.com/media/CUDeDoWUYAAD-EM.jpg,1,seat_belt,0.375057,False,miniature_pinscher,0.167175,True,Chihuahua,0.086951,True


In [27]:
sum(df_image['p1_conf'] >= 0.5)

1239

In [28]:
# consistency Data Frame
df_dog = df_image[df_image['p1_dog'] == True]


In [29]:
# 111 different dog breeds!
df_dog['p1'].value_counts()

golden_retriever      150
Labrador_retriever    100
Pembroke               89
Chihuahua              83
pug                    57
                     ... 
Scotch_terrier          1
EntleBucher             1
standard_schnauzer      1
Japanese_spaniel        1
groenendael             1
Name: p1, Length: 111, dtype: int64

to print full data frame from https://stackoverflow.com/questions/19124601/pretty-print-an-entire-pandas-series-dataframe

In [30]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df_dog['p1'].value_counts())

golden_retriever                  150
Labrador_retriever                100
Pembroke                           89
Chihuahua                          83
pug                                57
chow                               44
Samoyed                            43
toy_poodle                         39
Pomeranian                         38
cocker_spaniel                     30
malamute                           30
French_bulldog                     26
Chesapeake_Bay_retriever           23
miniature_pinscher                 23
German_shepherd                    20
Staffordshire_bullterrier          20
Siberian_husky                     20
Cardigan                           19
Eskimo_dog                         18
beagle                             18
Maltese_dog                        18
Shetland_sheepdog                  18
Shih-Tzu                           17
Lakeland_terrier                   17
Rottweiler                         17
Italian_greyhound                  16
kuvasz      

In [31]:
# haven't found different names for the same breed! we don't have consistency problem with this dataset

### Quality Issues ###

**1- Completeness:**

- Missing records: in this dataset we have 2075 entries, less than 2356, the number of entries in the downloaded by WeRateDogs dataset (checked above)
  
**2- Validity:**

- We have the same problem with the variable type of the column `tweet_id`. It should be a string instead an integer number 

**3 - Accuracy:**

- Not every predicted breed is a dog breed (e.g.: llama, guinea_pig). Some predictions are an object, not an animal (e.g.: seat_belt, shopping_cart, desktop_computer). This mistake happens in 543 predictions
- At `p1_conf` we have some poor predictions, (a breed has probability smaller than 50% - threshold choosen by me). The udacity neural network hasn't found a confident result for 836 records


### Tidiness Issues ###

- We should have an unique breed name in each row, in a column named `breed` as a variable. So we don't need the others columns (`p2_...` and `p3_...`)

### 3. Tweeter API downloaded Json file dataframe info ###

In [32]:
df_api.head()

,tweet_id,retweet_count,favorite_count
0,892420639486877696,7947,37149
1,892177413194625024,5902,31957
2,891815175371796480,3902,24066
3,891689552724799489,8099,40478
4,891327551943041024,8785,38706


In [33]:
df_api.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2333 entries, 0 to 2332
Data columns (total 3 columns):
tweet_id          2333 non-null object
retweet_count     2333 non-null int64
favorite_count    2333 non-null int64
dtypes: int64(2), object(1)
memory usage: 54.8+ KB


### Quality Issues ###

**1- Completeness:**

- Here the only problem is the mismatch in the number of entries. In this dataset we have 2333 entries against 2075 entries in the images data set and 2356 entries in the main dataset.

### Tidiness Issues ###

No tidiness issues in this dataset!

<a id='regression'></a>
### Part III - Cleaning Data

### Copy of the Data Frames ####

In [34]:
df_clean = df.copy()
df_image_clean = df_image.copy()
df_api_clean = df_api.copy()

### Quality Issues Cleaning ###

### Completeness ###

#### 1. Define ####

Remove `in_reply_to_status_id` , `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id`, `retweeted_status_timestamp` columns from df DataFrame.

#### Code ####

In [35]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [36]:
columns = ['in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 'retweeted_status_user_id', 
        'retweeted_status_timestamp']

def drop_columns(columns, df_clean):
    for column in columns:
        df_clean.drop(columns = column, axis= 1, inplace = True)

In [37]:
drop_columns(columns, df_clean)

#### Test ####

In [38]:
# Should have 12 columns instead 17
df_clean.shape[1] == 12

True

### Validity ###

#### Define ####

Fix Validity problems to df and df_image DataFrames using .astype(str) pandas command for `tweet_id` column and pd.to_datetime for `timestamp` column 

#### Code ####

In [39]:
df_clean['tweet_id'] = df_clean['tweet_id'].astype(str)

In [40]:
df_image_clean['tweet_id'] = df_image_clean['tweet_id'].astype(str)

In [41]:
df_clean['timestamp'] = pd.to_datetime(df_clean['timestamp'])

#### Test ####

In [42]:
# ok
df_clean.dtypes

tweet_id                           object
timestamp             datetime64[ns, UTC]
source                             object
text                               object
expanded_urls                      object
rating_numerator                    int64
rating_denominator                  int64
name                               object
doggo                              object
floofer                            object
pupper                             object
puppo                              object
dtype: object

In [43]:
# ok!
df_image_clean.dtypes

tweet_id     object
jpg_url      object
img_num       int64
p1           object
p1_conf     float64
p1_dog         bool
p2           object
p2_conf     float64
p2_dog         bool
p3           object
p3_conf     float64
p3_dog         bool
dtype: object

### Accuracy ###

#### Define ####

df_clean: 
- Drop the 442 rows where the numerator is smaller than the denominator
- Drop the 23 rows where the denominator is different from 10
- Create a new column `dog_rating` calculating `rating_numerator` / `rating_denominator`   

To drop these rows use df_clean.drop(row_index, inplace = True)

#### Code ####

In [44]:
df_clean.shape[0]

2356

In [45]:
# accuracy problems - 
wrong_numerator_index_number_array = df_clean[df_clean['rating_numerator'] < df_clean['rating_denominator']].index
wrong_numerator_index_number_array

Int64Index([  45,  229,  315,  342,  387,  462,  485,  599,  605,  730,
            ...
            2342, 2344, 2346, 2347, 2349, 2351, 2352, 2353, 2354, 2355],
           dtype='int64', length=442)

In [46]:
# drop rows
df_clean.drop(wrong_numerator_index_number_array, inplace = True)

In [47]:
wrong_denominator_index_number_array = df_clean[df_clean['rating_denominator'] != 10].index
wrong_denominator_index_number_array

Int64Index([ 313,  433,  516,  902, 1120, 1202, 1228, 1254, 1351, 1433, 1634,
            1635, 1663, 1779, 1843],
           dtype='int64')

In [48]:
# drop rows
df_clean.drop(wrong_denominator_index_number_array, inplace = True)

In [49]:
# create new column
df_clean['dog_rating'] = df_clean['rating_numerator'] / df_clean['rating_denominator']

#### Test ####

In [50]:
sum(df_clean['rating_numerator'] < df_clean['rating_denominator']) == 0

True

In [51]:
sum(df_clean['rating_denominator'] != 10) == 0 

True

In [53]:
df_clean['dog_rating'].sample(25)

1311    1.2
2281    1.0
2111    1.1
829     1.0
821     1.0
891     1.0
2178    1.0
142     1.2
255     1.2
937     1.2
503     1.2
2212    1.3
328     1.3
1345    1.3
2030    1.3
1301    1.0
1432    1.1
1884    1.1
357     1.2
1621    1.1
1235    1.0
1613    1.1
596     1.0
556     1.3
1048    1.0
Name: dog_rating, dtype: float64

#### Define ####

Drop rows where we don't have a dog breed and drop rows where the udacity neural networl algo didn't make a good prediction

#### Code ####

In [67]:
false_dog_breed_index = df_image_clean[df_image_clean['p1_dog'] == False].index
false_dog_breed_index

Int64Index([   6,    8,   17,   18,   21,   22,   25,   29,   33,   43,
            ...
            1992, 2012, 2013, 2021, 2022, 2026, 2046, 2052, 2071, 2074],
           dtype='int64', length=543)

In [68]:
df_image_clean.drop(false_dog_breed_index, inplace = True)

In [69]:
bad_dog_breed_prediction = df_image_clean[df_image_clean['p1_conf'] < 0.5].index
bad_dog_breed_prediction

Int64Index([   0,    3,    9,   12,   13,   14,   16,   19,   24,   26,
            ...
            2039, 2042, 2047, 2050, 2057, 2063, 2066, 2068, 2069, 2073],
           dtype='int64', length=559)

In [70]:
df_image_clean.drop(bad_dog_breed_prediction, inplace = True)

#### Test ####

In [71]:
sum(df_image_clean['p1_dog'] == False) == 0

True

In [72]:
sum(df_image_clean['p1_conf'] < 0.5) == 0

True

In [73]:
df_image_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 973 entries, 1 to 2072
Data columns (total 12 columns):
tweet_id    973 non-null object
jpg_url     973 non-null object
img_num     973 non-null int64
p1          973 non-null object
p1_conf     973 non-null float64
p1_dog      973 non-null bool
p2          973 non-null object
p2_conf     973 non-null float64
p2_dog      973 non-null bool
p3          973 non-null object
p3_conf     973 non-null float64
p3_dog      973 non-null bool
dtypes: bool(3), float64(3), int64(1), object(5)
memory usage: 78.9+ KB


In [74]:
# entries numbers dropped from 2075 to 973

### Tidiness ###

#### 1. Define ####